# Steepest descent to fit a Gaussian model

The *Gradient descent* algorithm descends a pre-defined function.  

Here we shall descend the $\chi^2$ (chi squared) function which is both:  
 - a function of the parameters that we are to optimise
 - the data that the model is to fit to

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from HeightsModule import *

## Background
If we have data for the heights of people in a population, it can be plotted as a histogram, i.e., a bar chart where each bar has a width representing a range of heights, and an area which is the probability of finding a person with a height in that range.
We can look to model that data with a function, such as a Gaussian, which we can specify with two parameters, rather than holding all the data in the histogram.

The Gaussian function is given as,
$$f(\mathbf{x};\mu, \sigma) = \frac{1}{\sigma\sqrt{2\pi}}\exp\left(-\frac{(\mathbf{x} - \mu)^2}{2\sigma^2}\right)$$

The figure above shows the data in orange, the model in magenta, and where they overlap in green.
This particular model has not been fit well - there is not a strong overlap.

Recall from the videos the definition of $\chi^2$ as the squared difference of the data and the model, i.e $\chi^2 = |\mathbf{y} - f(\mathbf{x};\mu, \sigma)|^2$. This is represented in the figure as the sum of the squares of the pink and orange bars.

Don't forget that $\mathbf{x}$ an $\mathbf{y}$ are represented as vectors here, as these are lists of all of the data points, the |*abs-squared*|${}^2$ encodes squaring and summing of the residuals on each bar.

To improve the fit, we will want to alter the parameters $\mu$ and $\sigma$, and ask how that changes the $\chi^2$.
That is, we will need to calculate the Jacobian,
$$ \mathbf{J} = \left[ \frac{\partial ( \chi^2 ) }{\partial \mu} , \frac{\partial ( \chi^2 ) }{\partial \sigma} \right]\;. $$

Let's look at the first term, $\frac{\partial ( \chi^2 ) }{\partial \mu}$, using the multi-variate chain rule, this can be written as,
$$ \frac{\partial ( \chi^2 ) }{\partial \mu} = -2 (\mathbf{y} - f(\mathbf{x};\mu, \sigma)) \cdot \frac{\partial f}{\partial \mu}(\mathbf{x};\mu, \sigma)$$
With a similar expression for $\frac{\partial ( \chi^2 ) }{\partial \sigma}$; try and work out this expression for yourself.

The Jacobians rely on the derivatives $\frac{\partial f}{\partial \mu}$ and $\frac{\partial f}{\partial \sigma}$.
Write functions below for these.

In [ ]:
# This is the Gaussian function
def f (x, mu, sig) :
    return np.exp(-(x-mu)**2/(2*sig**2)) / np.sqrt(2*np.pi) / sig

In [ ]:
# Next up, the derivative of f with respect to μ
def dfdmu (x, mu, sig) :
    return f(x, mu, sig) * ((x-mu)/(sig**2))

In [ ]:
# Finally in this cell, the derivative of f with respect to σ
def dfdsig (x, mu, sig) :
    f = 1 / sig
    g = 1 / np.sqrt(2*np.pi) * np.exp(-(x-mu)**2/(2*sig**2))
    f1 = -1 / sig**2
    g1 = g * ((x-mu)**2 / sig**2) * f
    return f*g1 + f1*g  # f(x, mu, sig) * ((x-mu)**2/(sig**3)) * (-1/sig)

Next recall that steepest descent shall move around in parameter space proportional to the negative of the Jacobian,
i.e., $\begin{bmatrix} \delta\mu \\ \delta\sigma \end{bmatrix} \propto -\mathbf{J} $, with the constant of proportionality being the *aggression* of the algorithm.

Modify the function below to include the $\frac{\partial ( \chi^2 ) }{\partial \sigma}$ term of the Jacobian, the $\frac{\partial ( \chi^2 ) }{\partial \mu}$ term has been included for you.

### Steepest descend algorithm

In [ ]:
def steepest_step (x, y, mu, sig, aggression):
    """aggression is step size?"""
    J = np.array([
        -2*(y - f(x, mu, sig)) @ dfdmu(x, mu, sig),
        -2*(y - f(x, mu, sig)) @ dfdsig(x, mu, sig)])
    step = -J * aggression
    return step

## Test your code

In [ ]:
# First get the heights data, ranges and frequencies
x, y = heights_data()

# Next we'll assign trial values for these.
mu, sig = 154, 7

# We'll keep a track of these so we can plot their evolution.
p = np.array([[mu, sig]])

fig, axes = plt.subplots(1, 3, figsize=(22, 9))

# Plot the histogram for our parameter guess
histogram(f, [mu, sig], axes[0])

# Do a few rounds of steepest descent.
for i in range(50) :
    dmu, dsig = steepest_step(x, y, mu, sig, 2000)
    mu += dmu
    sig += dsig
    p = np.append(p, [[mu,sig]], axis=0)
    
# Plot the path through parameter space.
contour(f, p, axes[1])

# Plot the final histogram.
histogram(f, [mu, sig], axes[2])

Note that the path taken through parameter space is not necesarily the most direct path, as with steepest descent we always move perpendicular to the contours.